In [5]:
import modelutils
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras import callbacks

In [6]:
tensorboard_callback = callbacks.TensorBoard()
histories = []

# Training VoxNet.

In [3]:
dataset_name = "20180717-1102-voxelgrid-dataset.p"

print("Loading dataset...")
(x_input_train, y_output_train), (x_input_test, y_output_test), dataset_parameters = pickle.load(open(dataset_name, "rb"))
print(dataset_parameters)

Loading dataset...
{'input_type': 'voxelgrid', 'output_targets': ['height', 'weight'], 'random_seed': 666, 'voxelgrid_target_shape': (32, 32, 32), 'voxel_size_meters': 0.1, 'voxelgrid_random_rotation': True, 'dataset_size_train': 6000, 'dataset_size_test': 1000}


In [10]:
input_shape = (32, 32, 32)
output_size = 2 
model = modelutils.create_voxnet_model_homepage(input_shape, output_size)
model.summary()

 # Compile the model.
model.compile(
        optimizer="rmsprop",
        loss="mse",
        metrics=["mae"]
    )

# Train the model.
history = model.fit(
    x_input_train, y_output_train,
    epochs=50,
    validation_data=(x_input_test, y_output_test),
    callbacks=[tensorboard_callback]
    )

histories.append(history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 32, 32, 32, 1)     0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 14, 14, 14, 32)    4032      
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 12, 12, 12, 32)    27680     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 6, 6, 6, 32)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 6912)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               884864    
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 258       
Total para

KeyboardInterrupt: 

# Training PointNet.

In [ ]:
dataset_name = "20180717-1236-pointcloud-dataset.p"

print("Loading dataset...")
(x_input_train, y_output_train), (x_input_test, y_output_test), dataset_parameters = pickle.load(open(dataset_name, "rb"))
print(dataset_parameters)

def transform(x_input, y_output):

    x_input_transformed = []
    y_output_transformed = []
    for input_sample, output_sample in zip(x_input_train, y_output_train):
        if input_sample.shape[0] == 30000:
            x_input_transformed.append(input_sample[:,0:3])
            y_output_transformed.append(output_sample)
        else:
            print(input_sample.shape)
            
    x_input_transformed = np.array(x_input_transformed)
    y_output_transformed = np.array(y_output_transformed)
    return x_input_transformed, y_output_transformed
    
x_input_train, y_output_train = transform(x_input_train, y_output_train)
x_input_test, y_output_test = transform(x_input_test, y_output_test)

print("Training data input shape:", x_input_train.shape)
print("Training data output shape:", y_output_train.shape)
print("Testing data input shape:", x_input_test.shape)
print("Testing data output shape:", y_output_test.shape)
print("")

In [9]:
input_shape = (30000, 3)
output_size = 2 
model = modelutils.create_point_net(input_shape, output_size)
model.summary()

 # Compile the model.
model.compile(
        optimizer="rmsprop",
        loss="mse",
        metrics=["mae"]
    )

# Train the model.
history = model.fit(
    x_input_train, y_output_train,
    epochs=50,
    validation_data=(x_input_test, y_output_test),
    callbacks=[tensorboard_callback]
    )

histories.append(history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30000, 3)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 30000, 3)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 30000, 64)         256       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30000, 64)         256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 30000, 64)         4160      
_________________________________________________________________
batch_normalization_7 (Batch (None, 30000, 64)         256       
_________________________________________________________________
lambda_2 (Lambda)            (None, 30000, 64)         0         
__________

TypeError: Unrecognized keyword arguments: {'callback': [<keras.callbacks.TensorBoard object at 0x119b0cb70>]}

# Plot the histories.

In [ ]:
for index, history in histories:
    model = models_to_train[i]
    for key, data in history.history.items():
        plt.plot(data, label=model.name + "-" + key)
plt.show()
plt.close()